# Lecture 1. Randomized Quicksort and Karger's min-cut

## Links:
### https://www.youtube.com/watch?v=sXHr3CDAeWE&list=PLOQjlWvnI0faRpH2oJcyW4CuM5Clt8a2n

## Randomized Quicksort

In [1]:
import copy
import random
from collections import defaultdict

In [2]:
def RQS(array):
    # O(nlogn) time (in expectation)
    # O(n) memory

    n = len(array)

    # base cases
    if n == 0:
        return []
    elif n == 1:
        return array
    elif n == 2:
        if array[0] <= array[1]:
            return array
        else:
            return array[::-1]
    else:
        # choose a pivot uar
        pivot_idx = random.randint(0, n-1)
        pivot = array[pivot_idx]
        L = []
        R = []

        # break array in two:
        # L for the elements < pivot
        # R for the elements >= pivot
        for i in range(n):
            if i != pivot_idx:
                if array[i] < pivot:
                    L.append(array[i])
                else:
                    R.append(array[i])
        
        # solve L and R recursively
        L = RQS(L)
        R = RQS(R)
        
        return L + [pivot] + R

### Tests

In [3]:
input0 = [0,1,2,3,4,5]
RQS(input0)

[0, 1, 2, 3, 4, 5]

In [4]:
input1 = [5,4,3,2,1,0]
RQS(input1)

[0, 1, 2, 3, 4, 5]

In [5]:
# test on a random sample of size=size, sampled from the integer interval [left_lim, right_lim-1]
size = 5
left_lim = 0
right_lim = 5
input2 = random.sample(range(left_lim, right_lim), size)
print(input2)

RQS(input2)

[2, 3, 1, 4, 0]


[0, 1, 2, 3, 4]

## Randomized min-cut (Karger's algorithm)

### Graph definition

In [6]:
# Definition of graph (non-directed connected weighted graph)
class graph:
    # O(m) space, m = # edges
    def __init__(self):
        self.size = 0
        self.weights = defaultdict(lambda: defaultdict(list))
        self.nodes = set()
        
    def add_edge(self, edge):
        # O(1) time and memory
        a = edge[0]
        b = edge[1]
        w = edge[2]
        
        # add nodes
        self.nodes.add(a)
        self.nodes.add(b)
        
        # add edge to edges
        self.weights[a][b].append(w)
        self.weights[b][a].append(w)
            
        # increase size
        self.size += 1
        
    def contract_edges(self, a, b):
        # O(m) time and O(1) memory
        
        # remove larger node
        contracted, to_remove = min(a,b), max(a,b)
        
        # decrease size
        self.size -= len(self.weights[to_remove][contracted])
        
        # remove node from nodes set
        self.nodes.remove(to_remove)
        
        # for every neighbor of the node to remove
        for node in self.weights[to_remove]:
            # transfer every edge to contracted edge
            if node != contracted:
                self.weights[contracted][node] += self.weights[to_remove][node]
                self.weights[node][contracted] += self.weights[to_remove][node]
                    
            # remove all edges to removing node
            del self.weights[node][to_remove]
                
        # remove all edges from removing node
        del self.weights[to_remove]
    
    def print_state(self):
        # O(1) time and memory
        print(self.size)
        print(self.nodes)
        print(self.weights)
        
    def get_weight(self):
        # O(m) time and O(1) memory
        output = 0
        for u in self.weights:
            for v in self.weights[u]:
                output += sum(self.weights[u][v])
                
        return output/2.0
    
    def sample_edge(self):
        # O(1) time and memory
        # choose a node uar
        u = random.sample(self.nodes, 1)[0]
        
        # choose a second node uar from neighbors of u
        v = random.sample(self.weights[u].keys(), 1)[0]
        
        return [u,v]

#### test graph

In [7]:
G = graph()
G.print_state()

0
set()
defaultdict(<function graph.__init__.<locals>.<lambda> at 0x110b4ed30>, {})


In [8]:
G.add_edge([1,2,0.125])
G.add_edge([0,1,0.5])
G.add_edge([2,0,0.25])
G.print_state()

3
{0, 1, 2}
defaultdict(<function graph.__init__.<locals>.<lambda> at 0x110b4ed30>, {1: defaultdict(<class 'list'>, {2: [0.125], 0: [0.5]}), 2: defaultdict(<class 'list'>, {1: [0.125], 0: [0.25]}), 0: defaultdict(<class 'list'>, {1: [0.5], 2: [0.25]})})


In [9]:
# parallel edge
G.add_edge([1,0,1])
G.print_state()

4
{0, 1, 2}
defaultdict(<function graph.__init__.<locals>.<lambda> at 0x110b4ed30>, {1: defaultdict(<class 'list'>, {2: [0.125], 0: [0.5, 1]}), 2: defaultdict(<class 'list'>, {1: [0.125], 0: [0.25]}), 0: defaultdict(<class 'list'>, {1: [0.5, 1], 2: [0.25]})})


In [10]:
G.add_edge([1,3,2])
G.add_edge([3,2,4])
G.add_edge([4,3,8])
G.add_edge([4,2,16])
G.print_state()

8
{0, 1, 2, 3, 4}
defaultdict(<function graph.__init__.<locals>.<lambda> at 0x110b4ed30>, {1: defaultdict(<class 'list'>, {2: [0.125], 0: [0.5, 1], 3: [2]}), 2: defaultdict(<class 'list'>, {1: [0.125], 0: [0.25], 3: [4], 4: [16]}), 0: defaultdict(<class 'list'>, {1: [0.5, 1], 2: [0.25]}), 3: defaultdict(<class 'list'>, {1: [2], 2: [4], 4: [8]}), 4: defaultdict(<class 'list'>, {3: [8], 2: [16]})})


In [11]:
G.get_weight()

31.875

In [12]:
G.contract_edges(4,3)
G.print_state()

7
{0, 1, 2, 3}
defaultdict(<function graph.__init__.<locals>.<lambda> at 0x110b4ed30>, {1: defaultdict(<class 'list'>, {2: [0.125], 0: [0.5, 1], 3: [2]}), 2: defaultdict(<class 'list'>, {1: [0.125], 0: [0.25], 3: [4, 16]}), 0: defaultdict(<class 'list'>, {1: [0.5, 1], 2: [0.25]}), 3: defaultdict(<class 'list'>, {1: [2], 2: [4, 16]})})


In [13]:
G.sample_edge()

[3, 1]

In [14]:
sample = G.sample_edge()
print(sample)
G.contract_edges(sample[0], sample[1])
G.print_state()

[3, 1]
6
{0, 1, 2}
defaultdict(<function graph.__init__.<locals>.<lambda> at 0x110b4ed30>, {1: defaultdict(<class 'list'>, {2: [0.125, 4, 16], 0: [0.5, 1]}), 2: defaultdict(<class 'list'>, {1: [0.125, 4, 16], 0: [0.25]}), 0: defaultdict(<class 'list'>, {1: [0.5, 1], 2: [0.25]})})


In [15]:
sample = G.sample_edge()
print(sample)
G.contract_edges(sample[0], sample[1])
G.print_state()

[0, 2]
5
{0, 1}
defaultdict(<function graph.__init__.<locals>.<lambda> at 0x110b4ed30>, {1: defaultdict(<class 'list'>, {0: [0.5, 1, 0.125, 4, 16]}), 0: defaultdict(<class 'list'>, {1: [0.5, 1, 0.125, 4, 16]})})


In [16]:
G.get_weight()

21.625

### Karger's algorithm

In [17]:
def contract(G_0):
    # O(nm) time and O(m) memory, m = # edges
    G = copy.deepcopy(G_0)
    
    # contract until only 2 nodes left
    while len(G.nodes) > 2:
        a,b = G.sample_edge()
        G.contract_edges(a,b)
        
    return G.get_weight()

def Karger(G):
    # O(nm^3) time and O(m^2) memory, m = # edges
    G.print_state()
    
    c = 3 # (P(fail) <= e^(-c))
    m = G.size
    ans = G.get_weight()
    ans_list = []
    
    # repeat cm^2 times: contract until only 2 nodes left
    # and return best output
    for _ in range(c*(m**2)):
        ans_list.append(contract(G))
        ans = min(ans, ans_list[-1])
        
    return ans, ans_list

### Tests

#### INPUT 0

In [18]:
G = graph()

G.add_edge([1,2,0.125])
G.add_edge([0,1,0.5])
G.add_edge([2,0,0.25])

# parallel edge
G.add_edge([1,0,1])

G.add_edge([1,3,2])
G.add_edge([3,2,4])
G.add_edge([4,3,8])
G.add_edge([4,2,16])
G.print_state()

8
{0, 1, 2, 3, 4}
defaultdict(<function graph.__init__.<locals>.<lambda> at 0x110b76af0>, {1: defaultdict(<class 'list'>, {2: [0.125], 0: [0.5, 1], 3: [2]}), 2: defaultdict(<class 'list'>, {1: [0.125], 0: [0.25], 3: [4], 4: [16]}), 0: defaultdict(<class 'list'>, {1: [0.5, 1], 2: [0.25]}), 3: defaultdict(<class 'list'>, {1: [2], 2: [4], 4: [8]}), 4: defaultdict(<class 'list'>, {3: [8], 2: [16]})})


In [19]:
Karger(G)

8
{0, 1, 2, 3, 4}
defaultdict(<function graph.__init__.<locals>.<lambda> at 0x110b76af0>, {1: defaultdict(<class 'list'>, {2: [0.125], 0: [0.5, 1], 3: [2]}), 2: defaultdict(<class 'list'>, {1: [0.125], 0: [0.25], 3: [4], 4: [16]}), 0: defaultdict(<class 'list'>, {1: [0.5, 1], 2: [0.25]}), 3: defaultdict(<class 'list'>, {1: [2], 2: [4], 4: [8]}), 4: defaultdict(<class 'list'>, {3: [8], 2: [16]})})


(1.75,
 [24.0,
  24.0,
  1.75,
  24.0,
  2.375,
  14.0,
  14.0,
  20.375,
  14.0,
  14.0,
  22.0,
  20.375,
  22.0,
  14.0,
  2.375,
  1.75,
  22.0,
  12.375,
  21.625,
  24.0,
  3.625,
  1.75,
  24.0,
  24.0,
  3.625,
  3.625,
  14.0,
  24.0,
  2.375,
  20.375,
  22.0,
  3.625,
  3.625,
  1.75,
  20.375,
  24.0,
  13.625,
  22.0,
  1.75,
  2.375,
  3.625,
  2.375,
  3.625,
  22.0,
  24.0,
  13.625,
  24.0,
  20.375,
  14.0,
  1.75,
  12.375,
  3.625,
  14.0,
  2.375,
  12.375,
  21.625,
  12.375,
  22.0,
  2.375,
  21.625,
  22.0,
  3.625,
  13.625,
  22.0,
  24.0,
  22.0,
  14.0,
  1.75,
  22.0,
  22.0,
  22.0,
  24.0,
  24.0,
  1.75,
  3.625,
  1.75,
  24.0,
  24.0,
  24.0,
  2.375,
  20.375,
  1.75,
  24.0,
  24.0,
  22.0,
  1.75,
  22.0,
  1.75,
  2.375,
  1.75,
  3.625,
  2.375,
  22.0,
  1.75,
  2.375,
  12.375,
  21.625,
  14.0,
  2.375,
  13.625,
  1.75,
  21.625,
  2.375,
  12.375,
  22.0,
  3.625,
  20.375,
  21.625,
  21.625,
  2.375,
  12.375,
  14.0,
  13.625,
  1.75,
  1